## Calcular o Tempo de Voto

## Importing libraries

In [27]:
import duckdb
import pandas as pd
import time

## Importing Data

In [28]:
cursor = duckdb.connect()

In [29]:
TABLE = "read_parquet('UFS_VOTE_EVENTS.parquet/*/*/*.parquet', hive_partitioning=True)"

In [30]:
source_data = f"""
    (
        SELECT
        *
        FROM {TABLE}
    ) AS source
"""


## Preparinga Data

### Criando um ID único para cada voto

Como heurística, vamos criar um id único para cada voto, que será determinado a partir de uma operação âncora.

A operação servirá como marcação de que um voto foi iniciado e, todas as linhas entre uma operação âncora e a próxima, serão consideradas como um único voto.

In [31]:
ANCHOR_OPERATION = 'Aguardando digitação do título'

In [32]:
query_create_id = f"""
    (
        SELECT
            *,
            SUM(CASE WHEN event_description = '{ANCHOR_OPERATION}' THEN 1 ELSE 0 END) 
            OVER (PARTITION BY event_date, uf, filename ORDER BY event_timestamp) AS vote_id
        FROM {source_data}
    ) AS query_vote_id
"""

### Pivotando timestamp dos eventos por id

In [36]:
timestamp_inicio_fim_voto = [
    f'''
        MAX(
            CASE WHEN event_description = 'Título digitado pelo mesário' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_titulo_digitado,
    ''',
    f'''
        MAX(
            CASE WHEN event_description = 'O voto do eleitor foi computado' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_voto_computado
    '''
]

In [33]:
VOTE_EVENTS = [
    'Voto confirmado para [Conselheiro Distrital]',
    'Voto confirmado para [Deputado Distrital]',
    'Voto confirmado para [Deputado Estadual]',
    'Voto confirmado para [Deputado Federal]',
    'Voto confirmado para [Governador]',
    'Voto confirmado para [Prefeito]',
    'Voto confirmado para [Presidente]',
    'Voto confirmado para [Senador]',
]

timestamp_vote_events = [
    f'''
        MAX(
            CASE WHEN event_description = \'{event}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_voto_{event.replace("Voto confirmado para [", "").replace("]", "").lower()}
    '''
    for event in VOTE_EVENTS
]

In [35]:
BIOMETRIA_TENTATIVAS = [
    'Solicita digital. Tentativa [1] de [4]',
    'Solicita digital. Tentativa [2] de [4]',
    'Solicita digital. Tentativa [3] de [4]',
    'Solicita digital. Tentativa [4] de [4]',
    'Solicitação de dado pessoal do eleitor para habilitação manual',
    'Eleitor foi habilitado'
]

timestamp_biometria_tentativas = [
    f'''
        MAX(
            CASE WHEN event_description = \'{event}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_biometria_{event.replace("Solicita digital. Tentativa [", "").replace("] de [4]", "").lower()}
    '''
    for event in BIOMETRIA_TENTATIVAS
    if event != 'Solicitação de dado pessoal do eleitor para habilitação manual'
] + [
    f'''
        MAX(
            CASE WHEN event_description = \'{BIOMETRIA_TENTATIVAS[-2]}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_biometria_manual
    '''
] + [
    f'''
        MAX(
            CASE WHEN event_description = \'{BIOMETRIA_TENTATIVAS[-1]}\' THEN event_timestamp ELSE NULL END 
        ) AS timestamp_habilitacao_eleitor
    '''
]
    

In [ ]:
query_pivot_timestamps = f"""(
    SELECT
        event_date, uf, filename, vote_id,
        
        MAX(city_code) AS city_code,
        MAX(zone_code) AS zone_code,
        MAX(section_code) AS section_code,

        SUM( (event_description='O voto do eleitor foi computado')::INT ) AS quantidade_votos_computados,
        SUM( (event_description ILIKE 'Solicita digital%')::INT ) AS quantidade_solicitacoes_biometria,
        SUM( (event_description ILIKE 'Voto confirmado para%')::INT ) AS quantidade_cargos_votados,
        MAX( (event_description='Solicitação de dado pessoal do eleitor para habilitação manual')::INT ) AS biometria_nao_funcionou,

        MIN( event_timestamp ) AS timestamp_primeiro_evento,

        {', '.join(timestamp_vote_events)},
        {', '.join(timestamp_biometria_tentativas)},
        {', '.join(timestamp_inicio_fim_voto)}
        
    FROM {query_create_id}
    GROUP BY event_date, uf, filename, vote_id
) AS query_pivot_timestamps
"""

### Salvando resultados - Parquet

In [ ]:
query = F"""
    COPY ({query_pivot_timestamps}) TO 'VOTES.parquet' (FORMAT 'parquet', PARTITION_BY (event_date, uf), OVERWRITE_OR_IGNORE 1);
"""

In [ ]:
tic = time.time()
cursor.execute(query)
toc = time.time()

print(F"Time {toc - tic}s")